<a href="https://colab.research.google.com/github/ferasshita/AI-Projects/blob/main/QPE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instructions

The places where you have enter code, to answer the questions, are marked with `# YOUR CODE HERE`. Once you have written your code you should remove the `raise NotImplementedError()` statement.

The first two questions are on phase estimation and the rest is about order finding algorithm.

## Question 1 (2 points)

Complete the function `create_operator` that returns a controlled $U$ gate where $U$ is an operator with eigenvector $|1\rangle$ and eigenvalue $e^{2\pi i \phi}$.

The function `create_operator` has

- Input: $\phi$
- Returns: Controlled $U$ gate with the properties described above

Don't create any circuit. Only a gate should be returned.

In [39]:
from cirq import CZPowGate

def create_operator(phi):

    CU = CZPowGate(exponent=2*phi)

    # Do not modify anything below this line
    return CU

In [40]:
# You can use this code to test your function by checking the bottom right corner of the unitary matrix
import cirq
from cmath import exp, pi

def test(phi):
    CU = create_operator(phi)
    unitary_matrix = cirq.unitary(CU)
    print(unitary_matrix)

phi=3/5
test(phi)
print(exp(2*pi*phi*1j))

[[ 1.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          1.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          1.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.        +0.j
  -0.80901699-0.58778525j]]
(-0.8090169943749476-0.587785252292473j)


In [41]:
# hidden tests in this cell will be used for grading.

In [42]:
# hidden tests in this cell will be used for grading.

## Question 2 (8 points)

Complete the function `estimate_phi` such that given a controlled $U$ gate where $U$ is an operator with eigenvector $|1\rangle$ and eigenvalue $e^{2\pi i \phi}$, it estimates and returns $\phi$.

The function `estimate_phi` has

- Input: Controlled $U$ gate
- Returns: Estimate for phi

You are given iqft and qpe algorithms and you can use them in your solution. Let the size of the first register equal 10.

In [32]:
def iqft(n,qubits,circuit):

    #Swap the qubits
    for i in range(n//2):
        circuit.append(SWAP(qubits[i],qubits[n-i-1]), strategy = InsertStrategy.NEW)

    #For each qubit
    for i in range(n-1,-1,-1):
        #Apply CR_k gates where j is the control and i is the target
        k=n-i #We start with k=n-i
        for j in range(n-1,i,-1):
            #Define and apply CR_k gate
            crk = CZPowGate(exponent = -2/2**(k))
            circuit.append(crk(qubits[j],qubits[i]),strategy = InsertStrategy.NEW)
            k=k-1 #Decrement at each step

        #Apply Hadamard to the qubit
        circuit.append(H(qubits[i]),strategy = InsertStrategy.NEW)

def qpe(t,control, target, circuit, CU):

    #Apply Hadamard to control qubits
    circuit.append(cirq.H.on_each(control))

    #Apply CU gates
    for i in range(t):
        #Obtain the power of CU gate
        CUi = CU**(2**i)
        #Apply CUi gate where t-i-1 is the control
        circuit.append(CUi(control[t-i-1],*target))

    #Apply inverse QFT
    iqft(t,control,circuit)

In [33]:

import cirq
from cirq.circuits import InsertStrategy
from cirq import H, SWAP, CZPowGate, X, measure

def estimate_phi(mystery):
    """
    Estimates the phase phi of a controlled U gate using the QPE algorithm.

    Args:
        mystery: The controlled U gate.

    Returns:
        The estimated value of phi.
    """

    # Create circuit
    circuit = cirq.Circuit()

    # Set the size of the control register
    t = 10

    # Create the control and target qubits
    control_qubits = cirq.LineQubit.range(t)
    target_qubit = cirq.LineQubit(t)

    # Initialize the target qubit to the eigenvector |1>
    circuit.append(X(target_qubit))

    # Apply the Quantum Phase Estimation algorithm
    qpe(t, control_qubits, [target_qubit], circuit, mystery)

    # Measure the control register
    circuit.append(measure(*control_qubits, key='result'))

    s=cirq.Simulator()
    samples=s.run(circuit, repetitions=1000)

    #Most frequent observation
    freq = list(samples.histogram(key='result').keys())[0]

    return freq/2**t

In [37]:
import math
# You can use this code to test your function by different operators
def test_qpe(phi):
    operator = CZPowGate(exponent=2*phi)
    return estimate_phi(operator)

assert(math.isclose(test_qpe(0.23),0.23,rel_tol=1e-2))

In [35]:
# hidden tests will be used for grading.

## Question 3 (1 point)

Complete the function `calcL` such that given $N$, it returns the size $L$ of the target register. Note that $L = \log_2 N$.

The function `calcL` has

- Input: $N$
- Returns: size of the target register

In [17]:

import math
def calcL(N):

    # L is the number of qubits required to represent N, which is ceil(log2(N)).
    L = math.ceil(math.log2(N))
    return int(L)

In [18]:
# You can use this cell to test your function

print(calcL(30))
print(calcL(62))

5
6


In [19]:
# hidden tests will be used for grading.

## Question 4 (1 point)

Complete the function `calct` such that given $N$, it returns the size $t$ of the control register. Note that $t = 2L + 1 + \big \lceil \log \big( 2 + \frac{1}{2\epsilon} \big) \big \rceil$. Let $\epsilon=0.1$.

The function `calct` has

- Input: $N$
- Returns: size of the control register

In [21]:

import math
def calct(N):
    """
    Calculates the size of the control register t.

    Args:
        N: The number to factor.

    Returns:
        The integer size t of the control register.
    """
    epsilon = 0.1
    # First, calculate L, the size of the target register.
    L = math.ceil(math.log2(N))
    # Then, calculate the additional term for precision.
    precision_term = math.ceil(math.log2(2 + 1 / (2 * epsilon)))
    # Calculate t using the formula.
    t = 2 * L + 1 + precision_term
    return int(t)

In [22]:
# You can use this cell to test your function

print(calct(30)) #For 30 the answer is 14
print(calct(62)) #For 62 the answer is 16

14
16


In [23]:
# hidden tests will be used for grading.

## Question 5 ( 2 points)

Complete the function `create_regs` such that given the size of the control and target registers, it returns the control and target registers.

The function `create_regs` has

- Input: $L$ and $t$
- Returns: Two registers `control` and `target`

In [43]:

import cirq
def create_regs(t,L):

    # The control register contains qubits 0 to t-1.
    control = cirq.LineQubit.range(t)
    # The target register contains qubits t to t+L-1.
    target = cirq.LineQubit.range(t, t + L)
    return control, target

In [25]:
# hidden tests will be used for grading.
# If this cell results in an error, your implementation is incorrect

In [26]:
# hidden tests will be used for grading.
# If this cell results in an error, your implementation is incorrect

## Question 6 (8 points)

Complete the function `order` so that given $x$ and $N$, it returns the histogram of samples from the order finding circuit.

The function `order` has

- Input: $x$ and $N$
- Returns: A histogram of samples

Notes:

- You are given a function named $U_x$ which implements $U_x |y\rangle \rightarrow |xy {\mod N}\rangle $ and returns its controlled version. Run the following cell to load the function. In order to use the function you should pass $x$ and $N$ as parameter.

    <pre>CU=Ux(x,N)</pre>

- You are also given the code for `qpe` and `iqft`.

In [28]:
import numpy as np
def Ux(x,N):

    k=1
    while(N>2**k):
        k=k+1

    u = np.zeros([2**k, 2**k], dtype = int)

    for i in range(N):
        u[x*i%N][i]=1
    for i in range(N,2**k):
        u[i][i]=1


    XU = cirq.MatrixGate(u).controlled()
    return XU

In [29]:
import cirq
from cirq.circuits import InsertStrategy
from cirq import H, SWAP, CZPowGate
def iqft(n,qubits,circuit):

    #Swap the qubits
    for i in range(n//2):
        circuit.append(SWAP(qubits[i],qubits[n-i-1]), strategy = InsertStrategy.NEW)

    #For each qubit
    for i in range(n-1,-1,-1):
        #Apply CR_k gates where j is the control and i is the target
        k=n-i #We start with k=n-i
        for j in range(n-1,i,-1):
            #Define and apply CR_k gate
            crk = CZPowGate(exponent = -2/2**(k))
            circuit.append(crk(qubits[j],qubits[i]),strategy = InsertStrategy.NEW)
            k=k-1 #Decrement at each step

        #Apply Hadamard to the qubit
        circuit.append(H(qubits[i]),strategy = InsertStrategy.NEW)

def qpe(t,control, target, circuit, CU):

    #Apply Hadamard to control qubits
    circuit.append(cirq.H.on_each(control))

    #Apply CU gates
    for i in range(t):
        #Obtain the power of CU gate
        CUi = CU**(2**i)
        #Apply CUi gate where t-i-1 is the control
        circuit.append(CUi(control[t-i-1],*target))

    #Apply inverse QFT
    iqft(t,control,circuit)

In [30]:

import cirq
from cirq import X, SWAP

def order(x,N):
    """
    Constructs and simulates the order-finding circuit for x mod N.

    Args:
        x: The base of the modular exponentiation.
        N: The modulus.

    Returns:
        A cirq.ResultDict histogram of the measurement outcomes.
    """
    #Create a circuit
    circuit = cirq.Circuit()

    # Determine the required number of qubits for the control and target registers
    L = calcL(N)
    t = calct(N)

    # Create the control and target registers
    control, target = create_regs(t, L)

    # Get the controlled unitary operator for modular multiplication
    CU = Ux(x, N)

    # Initialize the target register to the state |1>
    # This is done by applying an X gate to the least significant qubit of the target register.
    circuit.append(X(target[-1]))

    # Apply the Quantum Phase Estimation subroutine
    qpe(t, control, target, circuit, CU)

    #Measure the control register
    circuit.append(cirq.measure(*control, key='result'))

    #Sample the circuit
    s=cirq.Simulator()
    samples=s.run(circuit, repetitions=1000)

    # Return a histogram of samples
    results= samples.histogram(key='result')
    return results

In [31]:
#You can use this cell to test your function

results = order(4,31)
print(results)

Counter({3277: 196, 0: 189, 13107: 165, 9830: 116, 6554: 112, 9831: 63, 6553: 47, 13108: 12, 3276: 11, 13106: 10, 6552: 9, 9829: 8, 6555: 7, 9833: 6, 13109: 4, 9832: 4, 9827: 3, 6551: 3, 9841: 3, 9828: 3, 13110: 2, 9834: 2, 9773: 1, 3272: 1, 6548: 1, 6609: 1, 3330: 1, 3275: 1, 6543: 1, 9825: 1, 13333: 1, 6563: 1, 13097: 1, 3287: 1, 6546: 1, 6561: 1, 6557: 1, 6559: 1, 6547: 1, 13103: 1, 6550: 1, 3278: 1, 3279: 1, 9846: 1, 13111: 1, 13091: 1, 13171: 1})


In [44]:
# hidden tests will be used for grading.
# If this cell results in an error, your implementation is incorrect